# IBM Applied Data Science Capstone Project : The Battle of the Neighbourhoods

Week 4 and 5 of the Applied Data Science Capstone course.

In this project we are going to find out which are the best places to lives in Kochi.

## 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## 2. Scrapping data

In [2]:
data = requests.get('http://pincode.city/india/kerala/ernakulam/kochi').text

In [3]:
soup = BeautifulSoup(data, 'lxml')

After finding the table and table data create a dictionary called cell having 3 keys PostalCode, Location and City.

Append the data to the list.

In [4]:
kochitable = soup.find('table',class_ = 'table table-striped table-bordered')

In [5]:
print(kochitable)

<table class="table table-striped table-bordered">
<thead>
<tr>
<th>S.No</th>
<th>Pincode</th>
<th>Location</th>
<th>City</th>
</tr>
</thead>
<tbody>
<tr> <td>1</td><td><a class="pincode" href="/pincode/682008">682008</a></td><td><a class="" href="/india/kerala/ernakulam/kochi/andikkadavu/682008" title="Ernakulam">Andikkadavu</a></td><td><a class="city_list" href="/india/kerala/ernakulam/kochi">Kochi</a></td></tr><tr> <td>2</td><td><a class="pincode" href="/pincode/682501">682501</a></td><td><a class="" href="/india/kerala/ernakulam/kochi/ayyampilly/682501" title="Ernakulam">Ayyampilly</a></td><td><a class="city_list" href="/india/kerala/ernakulam/kochi">Kochi</a></td></tr><tr> <td>3</td><td><a class="pincode" href="/pincode/682508">682508</a></td><td><a class="" href="/india/kerala/ernakulam/kochi/azheekal/682508" title="Ernakulam">Azheekal</a></td><td><a class="city_list" href="/india/kerala/ernakulam/kochi">Kochi</a></td></tr><tr> <td>4</td><td><a class="pincode" href="/pincode/6820

In [6]:
table_contents = []

for k in kochitable.find_all('tbody'):
    rows = k.find_all('tr')
    for row in rows:
        cell = {}
        cell["Pincode"] = row.find(class_ = "pincode").text
        cell["City"] = row.find(class_ = "city_list").text
        cell["Neighbourhood"] = row.find(title = "Ernakulam").text
        table_contents.append(cell)
    
df=pd.DataFrame(table_contents)

In [7]:
print(df)

   Pincode   City       Neighbourhood
0   682008  Kochi         Andikkadavu
1   682501  Kochi          Ayyampilly
2   682508  Kochi            Azheekal
3   682008  Kochi           Chellanam
4   682010  Kochi            Edakochi
5   682502  Kochi          Edavanakad
6   682503  Kochi      Elamkunnapuzha
7   682008  Kochi           Kannamali
8   682001  Kochi               Kochi
9   682004  Kochi    Kochi Naval Base
10  682007  Kochi          Kumbalangi
11  682007  Kochi    Kumbalangi South
12  682511  Kochi           Malipuram
13  682029  Kochi          Matsyapuri
14  682002  Kochi        Mattancherry
15  682002  Kochi  Mattancherry Jetty
16  682002  Kochi   Mattancherry Town
17  682507  Kochi          Mundamveli
18  682505  Kochi            Narakkal
19  682509  Kochi        Nayarambalam
20  682507  Kochi             Nazreth
21  682509  Kochi            Nedungad
22  682009  Kochi           North End
23  682508  Kochi        Ochanthuruth
24  683515  Kochi           Palliport
25  682006  

In [8]:
df.shape

(32, 3)

In [9]:
df["Latitude"] = ""
df["Latitude"] = ""

There is some anomaly with the North End neighbourhood when we take the geocode in the next steps because it identifies North End in UK. We need North End in Kochi, Kerala, India. So we are going to edit it's name.

In [10]:
df = df.replace(to_replace ="North End",
                 value ="North End, Ernakulam ")

## 3. Finding the latitude and longitude of each pincode

In [11]:
# Need to drop those Neighborhood that the geocode does not find
to_drop_unknown = []
geolocator = Nominatim(user_agent="ny_explorer")
for index, row in df.iterrows():
    address = row['Neighbourhood'] + ', Kochi'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
        df.loc[index, 'Latitude'] = latitude
        df.loc[index, 'Longitude'] = longitude
    except AttributeError:
        print('Cannot do: {}, will drop index: {}'.format(address, index))
        to_drop_unknown.append(index)

Cannot do: Andikkadavu, Kochi, will drop index: 0
The geograpical coordinate of Ayyampilly, Kochi are 10.1263354, 76.1983715.
The geograpical coordinate of Azheekal, Kochi are 9.98943825, 76.24190933661936.
The geograpical coordinate of Chellanam, Kochi are 9.7912475, 76.2783617.
The geograpical coordinate of Edakochi, Kochi are 9.9128911, 76.2827473.
The geograpical coordinate of Edavanakad, Kochi are 10.0905883, 76.20683035822026.
The geograpical coordinate of Elamkunnapuzha, Kochi are 10.030615, 76.2220886.
Cannot do: Kannamali, Kochi, will drop index: 7
The geograpical coordinate of Kochi, Kochi are 9.931308, 76.2674136.
The geograpical coordinate of Kochi Naval Base, Kochi are 9.95584605, 76.27683358175503.
The geograpical coordinate of Kumbalangi, Kochi are 9.881573, 76.28751033712678.
The geograpical coordinate of Kumbalangi South, Kochi are 9.881573, 76.28751033712678.
The geograpical coordinate of Malipuram, Kochi are 10.019032, 76.22512415607815.
The geograpical coordinate of

In [12]:
print(df)

   Pincode   City          Neighbourhood   Latitude  Longitude
0   682008  Kochi            Andikkadavu                   NaN
1   682501  Kochi             Ayyampilly  10.126335  76.198371
2   682508  Kochi               Azheekal   9.989438  76.241909
3   682008  Kochi              Chellanam   9.791248  76.278362
4   682010  Kochi               Edakochi   9.912891  76.282747
5   682502  Kochi             Edavanakad  10.090588  76.206830
6   682503  Kochi         Elamkunnapuzha  10.030615  76.222089
7   682008  Kochi              Kannamali                   NaN
8   682001  Kochi                  Kochi   9.931308  76.267414
9   682004  Kochi       Kochi Naval Base   9.955846  76.276834
10  682007  Kochi             Kumbalangi   9.881573  76.287510
11  682007  Kochi       Kumbalangi South   9.881573  76.287510
12  682511  Kochi              Malipuram  10.019032  76.225124
13  682029  Kochi             Matsyapuri   9.939526  76.268163
14  682002  Kochi           Mattancherry   9.958766  76

Dropping items with null values.

In [13]:
df.dropna(subset = ["Latitude", "Longitude"], axis = 0, inplace = True)
df = df.reset_index(drop = True)

In [14]:
print(df)

   Pincode   City          Neighbourhood   Latitude  Longitude
0   682501  Kochi             Ayyampilly  10.126335  76.198371
1   682508  Kochi               Azheekal   9.989438  76.241909
2   682008  Kochi              Chellanam   9.791248  76.278362
3   682010  Kochi               Edakochi   9.912891  76.282747
4   682502  Kochi             Edavanakad  10.090588  76.206830
5   682503  Kochi         Elamkunnapuzha  10.030615  76.222089
6   682001  Kochi                  Kochi   9.931308  76.267414
7   682004  Kochi       Kochi Naval Base   9.955846  76.276834
8   682007  Kochi             Kumbalangi   9.881573  76.287510
9   682007  Kochi       Kumbalangi South   9.881573  76.287510
10  682511  Kochi              Malipuram  10.019032  76.225124
11  682029  Kochi             Matsyapuri   9.939526  76.268163
12  682002  Kochi           Mattancherry   9.958766  76.258513
13  682002  Kochi     Mattancherry Jetty   9.958301  76.260576
14  682002  Kochi      Mattancherry Town   9.958351  76

In [15]:
df.shape


(30, 5)

## 4. Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

## 5. Creating map of neighborhoods in Kochi

Let's first get the geographical address of Kochi.

In [17]:
address = 'Kochi, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kochi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kochi are 9.931308, 76.2674136.


Let's create the map with the neighbourhoods.

In [18]:
# create map of Manhattan using latitude and longitude values
map_Kochi = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'],df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Kochi)  
    
map_Kochi

In case the map is not loaded on github.

In [67]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/josephbenofficial/coursera-capstone/blob/master/Week%205/kochibeforecluster.PNG?raw=true")

## 6. Exploring the neighbourhood

Let's Explore the first neighbourhood & get the top 100 venues that are within a radius of 1km of the first neighbourhood.

In [19]:
df.loc[0,'Neighbourhood']

'Ayyampilly'

In [20]:
neighbourhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Ayyampilly are 10.1263354, 76.1983715.


In [21]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c9f95a94ff2c28109567ae'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 10.135335409000009,
    'lng': 76.20749685544689},
   'sw': {'lat': 10.117335390999992, 'lng': 76.1892461445531}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4efaa7f993ad7cf2ed4bfc90',
       'name': 'La Dame Rouge',
       'location': {'address': 'Manapilly, Ayyampally P.O.',
        'lat': 10.127742,
        'lng': 76.205018,
        'labeledLatLngs': [{'label': 'display

In [23]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-25-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,La Dame Rouge,Hotel,10.127742,76.205018
1,Ambedkar beach,Beach,10.119167,76.199829


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


Now let's do the same for all the neighbourhoods. First, we create a function to repeat the same process to all the neighborhoods in Kochi. Then we write a code to run the above function on each neighborhood and create a new dataframe called Kochi_venues.

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
Kochi_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
#note here I have used "Neighbourhood" as in the orignal dataframe it is like that but above function it does not have a "u".

Ayyampilly
Azheekal
Chellanam
Edakochi
Edavanakad
Elamkunnapuzha
Kochi
Kochi Naval Base
Kumbalangi
Kumbalangi South
Malipuram
Matsyapuri
Mattancherry
Mattancherry Jetty
Mattancherry Town
Mundamveli
Narakkal
Nayarambalam
Nazreth
Nedungad
North End, Ernakulam 
Ochanthuruth
Palliport
Palluruthy
Palluruthy South
Panayappilly
Puthuvype
S.Chellanam
Thoppumpady
Willingdon Island


In [29]:
print(Kochi_venues.shape)

(136, 7)


In [30]:
Kochi_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ayyampilly,10.126335,76.198371,La Dame Rouge,10.127742,76.205018,Hotel
1,Ayyampilly,10.126335,76.198371,Ambedkar beach,10.119167,76.199829,Beach
2,Azheekal,9.989438,76.241909,Vypin Light House,9.994002,76.238240,Beach
3,Azheekal,9.989438,76.241909,vypeen 2nd bridge,9.983216,76.243660,Harbor / Marina
4,Azheekal,9.989438,76.241909,Le Maritime,9.982712,76.241850,Hotel


In [31]:
Kochi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ayyampilly,2,2,2,2,2,2
Azheekal,4,4,4,4,4,4
Edakochi,5,5,5,5,5,5
Edavanakad,2,2,2,2,2,2
Elamkunnapuzha,3,3,3,3,3,3
Kochi,5,5,5,5,5,5
Kochi Naval Base,4,4,4,4,4,4
Kumbalangi,2,2,2,2,2,2
Kumbalangi South,2,2,2,2,2,2


Let's find out how many unique categories can be curated from all the returned venues.

In [32]:
print('There are {} uniques categories.'.format(len(Kochi_venues['Venue Category'].unique())))

There are 53 uniques categories.


## 7. Analyze each neighbourhood

In [33]:
# one hot encoding
Kochi_onehot = pd.get_dummies(Kochi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Kochi_onehot['Neighborhood'] = Kochi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Kochi_onehot.columns[-1]] + list(Kochi_onehot.columns[:-1])
Kochi_onehot = Kochi_onehot[fixed_columns]

Kochi_onehot.head()

,Neighborhood,ATM,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant
0,Ayyampilly,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ayyampilly,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Azheekal,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Azheekal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Azheekal,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
Kochi_onehot.shape

(136, 54)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [35]:
Kochi_grouped = Kochi_onehot.groupby('Neighborhood').mean().reset_index()
Kochi_grouped
Kochi_grouped.shape

(27, 54)

Let's print each neighborhood along with the top 5 most common venues.

In [36]:
num_top_venues = 5

for hood in Kochi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Kochi_grouped[Kochi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ayyampilly----
              venue  freq
0             Hotel   0.5
1             Beach   0.5
2               ATM   0.0
3        Lighthouse   0.0
4  Halal Restaurant   0.0


----Azheekal----
             venue  freq
0  Harbor / Marina  0.25
1            Hotel  0.25
2            Beach  0.25
3  Bed & Breakfast  0.25
4              ATM  0.00


----Edakochi----
        venue  freq
0         ATM   0.2
1  Playground   0.2
2        Café   0.2
3   Gift Shop   0.2
4         Gym   0.2


----Edavanakad----
              venue  freq
0  Asian Restaurant   0.5
1     Fishing Store   0.5
2               ATM   0.0
3        Lighthouse   0.0
4   Harbor / Marina   0.0


----Elamkunnapuzha----
                  venue  freq
0                 Beach  0.33
1  Fast Food Restaurant  0.33
2                  Lake  0.33
3                   ATM  0.00
4            Lighthouse  0.00


----Kochi----
              venue  freq
0  Halal Restaurant   0.2
1      Antique Shop   0.2
2               Bar   0.2
3   Bed & Break

Let's put that into a pandas dataframe.

First, let's write a function to sort the venues in descending order.
Second, let's create the new dataframe and display the top 10 venues for each neighborhood.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Kochi_grouped['Neighborhood']

for ind in np.arange(Kochi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Kochi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ayyampilly,Hotel,Beach,ATM,Lighthouse,Halal Restaurant,Harbor / Marina,Historic Site,Hotel Bar,Indian Restaurant,Indie Movie Theater
1,Azheekal,Harbor / Marina,Hotel,Beach,Bed & Breakfast,ATM,Lighthouse,Historic Site,Hotel Bar,Indian Restaurant,Indie Movie Theater
2,Edakochi,ATM,Playground,Café,Gift Shop,Gym,Scenic Lookout,Resort,Halal Restaurant,Harbor / Marina,Historic Site
3,Edavanakad,Asian Restaurant,Fishing Store,ATM,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater
4,Elamkunnapuzha,Beach,Fast Food Restaurant,Lake,ATM,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant


## 8. Business Problem

Now we are going to try to answer our two business problems:

1.	Which are the locations in Kochi with the best infrastructure facilities and thus probably the best locations for people to settle down?

2.	Which are the locations in Kochi with a bad infrastructure and thus the best options for investors looking for new opportunities?


In [39]:
Kochi_venues['Venue Category'].unique()

array(['Hotel', 'Beach', 'Harbor / Marina', 'Bed & Breakfast',
       'Gift Shop', 'Playground', 'Café', 'ATM', 'Gym', 'Fishing Store',
       'Asian Restaurant', 'Lake', 'Fast Food Restaurant',
       'Halal Restaurant', 'Bar', 'Antique Shop', 'Hotel Bar',
       'Indian Restaurant', 'Airport', 'Bus Stop', 'Kerala Restaurant',
       'Light Rail Station', 'Performing Arts Venue', 'Historic Site',
       'Boat or Ferry', 'Market', 'Convenience Store', 'Auditorium',
       'Bakery', 'Spa', 'Scenic Lookout', 'Fried Chicken Joint',
       'Bus Line', 'Flea Market', 'Platform', 'Bus Station', 'Food Court',
       'Food', 'Multiplex', 'Food & Drink Shop',
       'Vegetarian / Vegan Restaurant', 'Resort', 'Auto Garage',
       'Clothing Store', 'Indie Movie Theater', 'Department Store',
       'Smoke Shop', 'Lighthouse', 'Fish Market', 'Shopping Mall',
       'Jewelry Store', 'Mobile Phone Shop', 'Coffee Shop'], dtype=object)

Most of the facilities listed here are important for a healthy lifestyle. So we are not going to modify this list.

In [40]:
quality_kochi= Kochi_venues

In [41]:
quality_kochi

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ayyampilly,10.126335,76.198371,La Dame Rouge,10.127742,76.205018,Hotel
1,Ayyampilly,10.126335,76.198371,Ambedkar beach,10.119167,76.199829,Beach
2,Azheekal,9.989438,76.241909,Vypin Light House,9.994002,76.238240,Beach
3,Azheekal,9.989438,76.241909,vypeen 2nd bridge,9.983216,76.243660,Harbor / Marina
4,Azheekal,9.989438,76.241909,Le Maritime,9.982712,76.241850,Hotel
5,Azheekal,9.989438,76.241909,The Seaport Hotel,9.998035,76.239694,Bed & Breakfast
6,Edakochi,9.912891,76.282747,Aby Stores,9.914116,76.279882,Gift Shop
7,Edakochi,9.912891,76.282747,M K Arjunan Open Air Ground/Theater,9.914456,76.278700,Playground
8,Edakochi,9.912891,76.282747,black brew,9.915565,76.286975,Café
9,Edakochi,9.912891,76.282747,HDFC Bank ATM,9.914871,76.278140,ATM


In [42]:
quality_kochi_onehot = pd.get_dummies(quality_kochi[['Venue Category']], prefix="", prefix_sep="")

In [43]:
quality_kochi_onehot['Neighborhood'] = quality_kochi['Neighborhood'] 

In [44]:
fixed_columns = list(quality_kochi_onehot.columns[-1:]) + list(quality_kochi_onehot.columns[:-1])
quality_kochi_onehot = quality_kochi_onehot[fixed_columns]

In [45]:
quality_kochi_grouped = quality_kochi_onehot.groupby(["Neighborhood"]).sum().reset_index()
print(quality_kochi_grouped.shape)
quality_kochi_grouped.head()

(27, 54)


,Neighborhood,ATM,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant
0,Ayyampilly,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Azheekal,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Edakochi,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Edavanakad,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Elamkunnapuzha,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
quality_kochi_grouped['Total infrastructure'] =  quality_kochi_grouped[quality_kochi_grouped.drop(['Neighborhood'], axis=1).columns.values].sum(axis=1)

In [47]:
quality_kochi_grouped

,Neighborhood,ATM,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure
0,Ayyampilly,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,Azheekal,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
2,Edakochi,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5
3,Edavanakad,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,Elamkunnapuzha,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
5,Kochi,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
6,Kochi Naval Base,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
7,Kumbalangi,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
8,Kumbalangi South,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
9,Malipuram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### 1.	Which are the locations in Kochi with the best infrastructure facilities and thus probably the best locations for people to settle down?

In [48]:
quality_kochi_grouped[quality_kochi_grouped['Total infrastructure'] == quality_kochi_grouped['Total infrastructure'].max()].transpose()

,12
Neighborhood,Mattancherry Jetty
ATM,0
Airport,0
Antique Shop,0
Asian Restaurant,1
Auditorium,1
Auto Garage,0
Bakery,0
Bar,0
Beach,0


Mattancherry Jetty is the best place to settle, when you look at the infrastructure facilities.


In [49]:
quality_kochi_grouped[quality_kochi_grouped['Total infrastructure'] == quality_kochi_grouped['Total infrastructure'].min()].transpose()

,9,16
Neighborhood,Malipuram,Nayarambalam
ATM,0,0
Airport,0,0
Antique Shop,0,0
Asian Restaurant,0,0
Auditorium,0,0
Auto Garage,0,0
Bakery,0,0
Bar,0,0
Beach,0,0


The worst places to settle down are Malipuram, Mundamveli and Nayarambalam, when you look at infrastructure facilities.

In [50]:
quality_kochi_grouped

,Neighborhood,ATM,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure
0,Ayyampilly,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,Azheekal,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
2,Edakochi,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5
3,Edavanakad,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,Elamkunnapuzha,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
5,Kochi,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
6,Kochi Naval Base,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
7,Kumbalangi,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
8,Kumbalangi South,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
9,Malipuram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## 9. Cluster neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [51]:
kmerged = quality_kochi_grouped.copy()
kmerged = kmerged.join(df[["Pincode",'Latitude', 'Longitude', "Neighbourhood" ]].set_index("Neighbourhood"), on="Neighborhood")
fixed_columns = list(kmerged.columns[-3:]) + list(kmerged.columns[:-3])
kmerged = kmerged[fixed_columns]
kmerged

,Pincode,Latitude,Longitude,Neighborhood,ATM,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure
0,682501,10.126335,76.198371,Ayyampilly,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,682508,9.989438,76.241909,Azheekal,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
2,682010,9.912891,76.282747,Edakochi,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5
3,682502,10.090588,76.206830,Edavanakad,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,682503,10.030615,76.222089,Elamkunnapuzha,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
5,682001,9.931308,76.267414,Kochi,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
6,682004,9.955846,76.276834,Kochi Naval Base,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
7,682007,9.881573,76.287510,Kumbalangi,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
8,682007,9.881573,76.287510,Kumbalangi South,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
9,682511,10.019032,76.225124,Malipuram,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [52]:
# set number of clusters
kclusters = 5

Kochi_grouped_clustering = kmerged.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Kochi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 3, 2, 2, 3, 3, 3, 3, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Kochi_merged = kmerged

# merge Kochi_grouped with df (Kochi_data) to add latitude/longitude for each neighborhood
Kochi_merged = Kochi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Kochi_merged.head() # check the last columns!

,Pincode,Latitude,Longitude,Neighborhood,ATM,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,682501,10.126335,76.198371,Ayyampilly,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,Hotel,Beach,ATM,Lighthouse,Halal Restaurant,Harbor / Marina,Historic Site,Hotel Bar,Indian Restaurant,Indie Movie Theater
1,682508,9.989438,76.241909,Azheekal,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,Harbor / Marina,Hotel,Beach,Bed & Breakfast,ATM,Lighthouse,Historic Site,Hotel Bar,Indian Restaurant,Indie Movie Theater
2,682010,9.912891,76.282747,Edakochi,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5,3,ATM,Playground,Café,Gift Shop,Gym,Scenic Lookout,Resort,Halal Restaurant,Harbor / Marina,Historic Site
3,682502,10.090588,76.206830,Edavanakad,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,Asian Restaurant,Fishing Store,ATM,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater
4,682503,10.030615,76.222089,Elamkunnapuzha,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,Beach,Fast Food Restaurant,Lake,ATM,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant


In [54]:
print(Kochi_merged)

   Pincode   Latitude  Longitude           Neighborhood  ATM  Airport  \
0   682501  10.126335  76.198371             Ayyampilly    0        0   
1   682508   9.989438  76.241909               Azheekal    0        0   
2   682010   9.912891  76.282747               Edakochi    1        0   
3   682502  10.090588  76.206830             Edavanakad    0        0   
4   682503  10.030615  76.222089         Elamkunnapuzha    0        0   
5   682001   9.931308  76.267414                  Kochi    0        0   
6   682004   9.955846  76.276834       Kochi Naval Base    0        1   
7   682007   9.881573  76.287510             Kumbalangi    0        0   
8   682007   9.881573  76.287510       Kumbalangi South    0        0   
9   682511  10.019032  76.225124              Malipuram    0        0   
10  682029   9.939526  76.268163             Matsyapuri    0        0   
11  682002   9.958766  76.258513           Mattancherry    0        0   
12  682002   9.958301  76.260576     Mattancherry J

Drop the null values.

In [55]:
Kochi_merged.dropna(subset=['Cluster Labels'], inplace=True)

In [56]:
print(Kochi_merged)

   Pincode   Latitude  Longitude           Neighborhood  ATM  Airport  \
0   682501  10.126335  76.198371             Ayyampilly    0        0   
1   682508   9.989438  76.241909               Azheekal    0        0   
2   682010   9.912891  76.282747               Edakochi    1        0   
3   682502  10.090588  76.206830             Edavanakad    0        0   
4   682503  10.030615  76.222089         Elamkunnapuzha    0        0   
5   682001   9.931308  76.267414                  Kochi    0        0   
6   682004   9.955846  76.276834       Kochi Naval Base    0        1   
7   682007   9.881573  76.287510             Kumbalangi    0        0   
8   682007   9.881573  76.287510       Kumbalangi South    0        0   
9   682511  10.019032  76.225124              Malipuram    0        0   
10  682029   9.939526  76.268163             Matsyapuri    0        0   
11  682002   9.958766  76.258513           Mattancherry    0        0   
12  682002   9.958301  76.260576     Mattancherry J

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, tot, cluster in zip(Kochi_merged['Latitude'],Kochi_merged['Longitude'], Kochi_merged['Neighborhood'], Kochi_merged['Total infrastructure'], Kochi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + '& Total infrastructure:' + str(tot), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In case map does not load in github.

In [69]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/josephbenofficial/coursera-capstone/blob/master/Week%205/kochiaftercluster1.PNG?raw=true")

In [70]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/josephbenofficial/coursera-capstone/blob/master/Week%205/kochiaftercluster2.PNG?raw=true")

## 10. Examine clusters

In [58]:
Kochi_merged.loc[Kochi_merged['Cluster Labels'] == 0, Kochi_merged.columns[[1] + list(range(5, Kochi_merged.shape[1]))]]

,Latitude,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,9.958766,0,0,1,1,0,0,0,0,0,2,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,13,0,Café,Indian Restaurant,Boat or Ferry,Performing Arts Venue,Convenience Store,Market,Auditorium,Asian Restaurant,Historic Site,Hotel Bar
12,9.958301,0,0,1,1,0,0,0,0,0,2,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,3,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,17,0,Café,Indian Restaurant,Boat or Ferry,Hotel,Performing Arts Venue,Convenience Store,Market,Hotel Bar,Auditorium,Asian Restaurant
13,9.958351,0,0,1,1,0,0,0,0,0,2,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,13,0,Café,Indian Restaurant,Boat or Ferry,Performing Arts Venue,Convenience Store,Market,Auditorium,Asian Restaurant,Historic Site,Hotel Bar
23,9.947738,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9,0,Café,Smoke Shop,Auditorium,Hotel,Hotel Bar,Indie Movie Theater,Fast Food Restaurant,Department Store,Convenience Store,Light Rail Station


Cluster 1

In [59]:
Kochi_merged.loc[Kochi_merged['Cluster Labels'] == 1, Kochi_merged.columns[[1] + list(range(5, Kochi_merged.shape[1]))]]

,Latitude,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,10.169576,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,Beach,Clothing Store,ATM,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater


Cluster 2

In [60]:
Kochi_merged.loc[Kochi_merged['Cluster Labels'] == 2, Kochi_merged.columns[[1] + list(range(5, Kochi_merged.shape[1]))]]

,Latitude,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10.126335,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,Hotel,Beach,ATM,Lighthouse,Halal Restaurant,Harbor / Marina,Historic Site,Hotel Bar,Indian Restaurant,Indie Movie Theater
1,9.989438,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,Harbor / Marina,Hotel,Beach,Bed & Breakfast,ATM,Lighthouse,Historic Site,Hotel Bar,Indian Restaurant,Indie Movie Theater
3,10.090588,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,Asian Restaurant,Fishing Store,ATM,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater
4,10.030615,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,2,Beach,Fast Food Restaurant,Lake,ATM,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant
9,10.019032,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,Indian Restaurant,ATM,Gift Shop,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indie Movie Theater,Jewelry Store
14,9.925186,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,3,2,Spa,Scenic Lookout,Bakery,ATM,Light Rail Station,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar
15,10.042012,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,Bakery,Beach,Fast Food Restaurant,Lake,ATM,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar
16,10.063941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,2,Market,ATM,Light Rail Station,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater
17,10.101165,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,8,2,Fried Chicken Joint,Food Court,Food,Flea Market,Platform,Multiplex,Bus Line,Bus Station,Lake,Historic Site
19,10.001292,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,2,Café,Auto Garage,Resort,Beach,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant


Cluster 3

In [61]:
Kochi_merged.loc[Kochi_merged['Cluster Labels'] == 3, Kochi_merged.columns[[1] + list(range(5, Kochi_merged.shape[1]))]]

,Latitude,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,9.912891,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5,3,ATM,Playground,Café,Gift Shop,Gym,Scenic Lookout,Resort,Halal Restaurant,Harbor / Marina,Historic Site
5,9.931308,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3,Halal Restaurant,Antique Shop,Bar,Bed & Breakfast,Gym,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar
6,9.955846,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,3,Airport,Hotel,Hotel Bar,Indian Restaurant,Light Rail Station,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Indie Movie Theater
7,9.881573,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,Bus Stop,Kerala Restaurant,Lighthouse,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater
8,9.881573,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,Bus Stop,Kerala Restaurant,Lighthouse,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater
18,10.122497,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5,3,Vegetarian / Vegan Restaurant,Bakery,Food & Drink Shop,Indian Restaurant,Market,Scenic Lookout,Shopping Mall,Harbor / Marina,Spa,Historic Site
21,9.92121,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,4,3,Playground,ATM,Bakery,Light Rail Station,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant
22,9.92121,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,4,3,Playground,ATM,Bakery,Light Rail Station,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant
25,9.934813,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4,3,Gym,Harbor / Marina,Shopping Mall,Fish Market,Light Rail Station,Halal Restaurant,Historic Site,Hotel,Hotel Bar,Indian Restaurant
26,9.947789,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,3,Café,Jewelry Store,Mobile Phone Shop,Bus Station,Coffee Shop,Lighthouse,Harbor / Marina,Historic Site,Hotel,Hotel Bar


Cluster 4

In [62]:
Kochi_merged.loc[Kochi_merged['Cluster Labels'] == 4, Kochi_merged.columns[[1] + list(range(5, Kochi_merged.shape[1]))]]

,Latitude,Airport,Antique Shop,Asian Restaurant,Auditorium,Auto Garage,Bakery,Bar,Beach,Bed & Breakfast,Boat or Ferry,Bus Line,Bus Station,Bus Stop,Café,Clothing Store,Coffee Shop,Convenience Store,Department Store,Fast Food Restaurant,Fish Market,Fishing Store,Flea Market,Food,Food & Drink Shop,Food Court,Fried Chicken Joint,Gift Shop,Gym,Halal Restaurant,Harbor / Marina,Historic Site,Hotel,Hotel Bar,Indian Restaurant,Indie Movie Theater,Jewelry Store,Kerala Restaurant,Lake,Light Rail Station,Lighthouse,Market,Mobile Phone Shop,Multiplex,Performing Arts Venue,Platform,Playground,Resort,Scenic Lookout,Shopping Mall,Smoke Shop,Spa,Vegetarian / Vegan Restaurant,Total infrastructure,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,9.939526,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8,4,Café,Bar,Harbor / Marina,Light Rail Station,Halal Restaurant,Bed & Breakfast,Gym,Antique Shop,Scenic Lookout,Resort


This clustering is done on the basis of similarity between the Neighbourhoods. But in our business problem we have to cluster based on how good the infrastructure is. Now we are going to do that.

## 11. Observations

1. The middle part of Kochi has the most infrastructure development and is the best place to live.
2. The southern part is the second best choice to live.
3. The northern part is the worst choice to live.

## 12. Results

The following are the results obtained from this project:

1. Mattancherry Jetty is the best place to settle, when you look at the infrastructure facilities.
2. The worst places to settle down are Malipuram, Mundamveli and Nayarambalam, when you look at infrastructure facilities.


## 13. Conclusion

We have utilized the methods of data scraping, data cleaning and kmeans clustering along with utilities like Foursquare API and GeoPy to find out which are the best places to live in the city of Kochi when it comes to infrastructure facilities. 

There might be some anomalies in the overall analysis as Foursquare data for Kochi region is inadequate. So there is scope for further improvement.

### The End